In [10]:
!pip install langchain==0.0.301 openai==0.28.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires tokenizers<=0.20.3,>=0.13.2, but you have tokenizers 0.21.0 which is incompatible.
langchain-community 0.3.13 requires langchain<0.4.0,>=0.3.13, but you have langchain 0.0.301 which is incompatible.
langchain-community 0.3.13 requires langsmith<0.3,>=0.1.125, but you have langsmith 0.0.92 which is incompatible.
langchain-core 0.3.28 requires langsmith<0.3,>=0.1.125, but you have langsmith 0.0.92 which is incompatible.
langchain-openai 0.2.14 requires openai<2.0.0,>=1.58.1, but you have openai 0.28.0 which is incompatible.
ollama 0.4.4 requires httpx<0.28.0,>=0.27.0, but you have httpx 0.28.1 which is incompatible.



   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 2.8 MB/s eta 0:00:01
   ------------------------------ --------- 1.3/1.7 MB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 3.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: anyio
    Found existing installation: anyio 4.6.2.post1
    Uninstalling anyio-4.6.2.post1:
      Successfully uninstalled anyio-4.6.2.post1
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.147
    Uninstalling langsmith-0.1.147:
      Successfully uninstalled langsmith-0.1.147
  Attempting uninstall: openai
    Found existing installation: openai 1.58.1
    Uninstalling openai-1.58.1:
      Successfully uninstalled openai-1.58.1
  Attempting uninstall: langchain
    Found existin

In [20]:
import os
import math
from typing import TypedDict
from fastapi import FastAPI
from pydantic import BaseModel
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langgraph.graph import  StateGraph, START, END
from transformers import AutoTokenizer, AutoModel
import torch
import openai

In [21]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [22]:
class InputState(TypedDict):
    suggested_answer:str
    student_answer:str

class OutputState(TypedDict):
    sbert_score: float
    roberta_score:float
    distilroberts_score:float
    t5_score:float
    use_score:float
    gpt_score:float
    minilm_score:float
    electra_score:float
    labse_score:float
    feedback: str

class OverallState(InputState,OutputState):
    pass

In [23]:
def generate_prompt(suggested_answer: str, student_answer: str) -> str:
    return f"""
Suggested answer: "{suggested_answer}"
Student’s answer: "{student_answer}"

Evaluate how well the student’s answer matches the suggested answer on a scale from 0 to 10,
considering correctness, completeness, and clarity. Provide a numeric score followed by a one-line feedback.
Example: "8.5 - Good answer but slightly lacks depth."
"""

def llm_evaluation_node(state: InputState) -> OutputState:
    """Use OpenAI API to evaluate student answer and return llm_score with feedback."""
    prompt = generate_prompt(state["suggested_answer"], state["student_answer"])
    
    try:
        # Call OpenAI API
        response = openai.Completion.create(
            engine="text-davinci-003",  # Use appropriate OpenAI model
            prompt=prompt,
            max_tokens=100,
            temperature=0
        )
        output = response.choices[0].text.strip()

        # Extract score and feedback
        score, feedback = output.split(" - ", 1)
        score = float(score)
    except ValueError:
        score = 0.0
        feedback = "No valid feedback."
    except Exception as e:
        raise RuntimeError(f"Error during LLM evaluation: {e}")
    
    return {"llm_score": score, "llm_feedback": feedback}


In [25]:
!pip install openai==0.28

In [26]:
# Example input state
state = {
    "suggested_answer": "The capital of France is Paris.",
    "student_answer": "Paris is the capital of France."
}

# Test the Function
result = llm_evaluation_node(state)
print(result)

RuntimeError: Error during LLM evaluation: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
builder =StateGraph(OverallState,input=InputState,output=OutputState)
builder.add_node("")

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
import nest_asyncio
import uvicorn
import subprocess
from fastapi.responses import JSONResponse
from fastapi import FastAPI, HTTPException, status


In [ ]:
app = FastAPI()
class Item(BaseModel):
    suggested_answer:str
    student_answer:str
